In [ ]:
!pip install tensorflow_text node2vec sentence_transformers -q
!pip install tensorflow_addons
import tensorflow_addons as tfa
import pandas as pd
pd.options.mode.chained_assignment = None
from google.colab import drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from node2vec import Node2Vec
from sentence_transformers import SentenceTransformer
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Softmax
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import pickle
from gensim.models import Word2Vec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 38.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 7.4 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Labelled data
labelled=pd.read_csv('/content/drive/MyDrive/Dataset/UNGDC/labelled_data.csv')
labelled.drop('Unnamed: 0', axis=1, inplace=True)

# Unlabelled data
unlabelled=pd.read_csv('/content/drive/MyDrive/Dataset/UNGDC/unlabelled_data.csv')

# Removing labelled senteces from unlabelled dataset
mask = unlabelled['country'].isin(labelled['country']) & unlabelled['year'].isin(labelled['year']) & unlabelled['doc_id'].isin(labelled['doc_id'])
unlabelled = unlabelled[~mask]
unlabelled.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
class Preprocessor:
  '''
    Removes punctuations, numbers, whitespaces
    Converts sentences into lower case
  '''

  def __init__(self, numbers=True, punct=True, stopwords=True, empty_sentence=True):
    self.numbers = numbers
    self.punct = punct
    self.stopwords = stopwords
    self.empty_sentence = empty_sentence

  def apply(self, sentence):
    sentence = sentence.lower()
    if self.numbers:
      sentence = Preprocessor.remove_numbers(sentence)

    if self.punct: 
      sentence = Preprocessor.remove_punct(sentence)

    if self.stopwords: 
      sentence = Preprocessor.remove_stopwords(sentence)

    if self.empty_sentence: 
      sentence = Preprocessor.empty_sentence(sentence)

    if sentence!=None:
        sentence = re.sub(r'\s+', ' ', sentence)

    return sentence
  
  @staticmethod
  def remove_punct(sentence):
    sentence = re.sub(r'[^\w\s]', '', sentence)
    return sentence
  
  @staticmethod
  def remove_numbers(sentence):
    sentence = re.sub(r'[0-9]', '', sentence)
    return sentence

  @staticmethod
  def remove_stopwords(sentence):
    sentence_clean = ' '.join( [ word for word in sentence.split() if word.lower() not in set( stopwords.words('english') ) ] )
    return sentence_clean

  
  @staticmethod
  def empty_sentence(sentence):
    words=sentence.split()
    if (not all(elem == "" for elem in sentence)) and len(sentence)>2:
        return sentence
    else:
        return None

In [ ]:
processor = Preprocessor()

# Labelled Data
labelled_cleaned=labelled.copy()
labelled_cleaned['text_clean']=labelled_cleaned['text'].apply(lambda sentence: processor.apply(sentence))
labelled_cleaned.dropna(inplace=True)
labelled_cleaned.reset_index(inplace=True, drop=True)

# Unlabelled data
sample_size=2000

unlabelled_cleaned=unlabelled.sample(sample_size).copy()
unlabelled_cleaned['text_clean']=unlabelled_cleaned['text'].apply(lambda sentence: processor.apply(sentence))
unlabelled_cleaned.dropna(inplace=True)
unlabelled_cleaned.reset_index(inplace=True, drop=True)

In [ ]:
model_sentence_transformer = SentenceTransformer('sentence-transformers/bert-base-nli-mean-tokens')

In [ ]:
labelled_cleaned['embeddings']=labelled_cleaned['text_clean'].apply(model_sentence_transformer.encode)

unlabelled_cleaned['embeddings']=unlabelled_cleaned['text_clean'].apply(model_sentence_transformer.encode)

In [ ]:
labelled.to_csv('/content/drive/MyDrive/Dataset/UNGDC/labelled_cleaned.csv')

In [ ]:
unlabelled_cleaned=unlabelled.sample(200000).copy()
unlabelled_cleaned['text_clean']=unlabelled_cleaned['text'].apply(lambda sentence: processor.apply(sentence))
unlabelled_cleaned.dropna(inplace=True)
unlabelled_cleaned.reset_index(inplace=True, drop=True)

In [ ]:
unlabelled_cleaned.to_csv('/content/drive/MyDrive/Dataset/UNGDC/unlabelled_cleaned.csv')

In [ ]:
unlabelled_cleaned['embeddings']=unlabelled_cleaned['text_clean'].apply(model_sentence_transformer.encode)

KeyboardInterrupt: ignored

In [ ]:
unlabelled_cleaned.to_csv('/content/drive/MyDrive/Dataset/UNGDC/unlabelled_cleaned.csv')

In [ ]:
total_df=pd.concat([labelled_cleaned, unlabelled_cleaned])

In [ ]:
total_df 

,doc_id,text,country,year,coding,difficult,text_clean,embeddings
0,AUS_46_1991.txt.1,"I congratulate you, Sir, on your election as P...",AUS,1991,greeting,0.0,congratulate sir election president fortysixth...,"[-0.3700411, 0.8781355, 1.559748, 0.65238917, ..."
1,AUS_46_1991.txt.2,I am sure we will all benefit from your long e...,AUS,1991,greeting,0.0,sure benefit long experience united nations au...,"[-0.06405132, 0.7984151, 1.0481572, 0.26101378..."
2,AUS_46_1991.txt.3,Australia enjoys strong bonds of friendship wi...,AUS,1991,greeting,0.0,australia enjoys strong bonds friendship saudi...,"[0.15582064, 0.2770484, 0.698128, 0.39469266, ..."
3,AUS_46_1991.txt.4,Australia warmly welcomes to membership of thi...,AUS,1991,greeting,0.0,australia warmly welcomes membership body paci...,"[0.27420735, 0.2793362, -0.049841184, 0.288611..."
4,AUS_46_1991.txt.5,"Equally, as a country of the Asia-Pacific regi...",AUS,1991,greeting,0.0,equally country asiapacific region welcome arr...,"[-0.6073841, 0.67131954, 1.067805, 0.54470253,..."
...,...,...,...,...,...,...,...,...
1874,BWA_51_1996.txt.62,We salute the efforts of the regional leaders ...,BWA,1996,NaN,NaN,salute efforts regional leaders,"[-0.26406074, 0.16139503, 2.047986, 0.10644821..."
1875,BRB_74_2019.txt.22,I am therefore confident that the battle will ...,BRB,2019,NaN,NaN,therefore confident battle,"[-0.19107045, 0.042433403, 1.3055695, 0.587929..."
1876,TLS_69_2014.txt.428,need to have the courage to speak a language o...,TLS,2014,NaN,NaN,need courage speak language trust,"[-0.29551226, 0.31834635, 1.1873891, 0.4271717..."
1877,SEN_69_2014.txt.135,must be discouraged.,SEN,2014,NaN,NaN,must discouraged,"[-0.17290574, 0.12104951, 2.220055, 0.15633497..."


In [ ]:
edges_subset=list(combinations(total_df.index, 2))
edge_sim_labelled=[]
with open('/content/drive/MyDrive/Dataset/UNGDC/csim.csv', 'r+') as infile:
  for idx1, idx2 in tqdm(edges_subset):
    cosine_sim_val=cosine_similarity( [ total_df['embeddings'][idx1] ], [ total_df['embeddings'][idx2] ])
    infile.writeline(f"{total_df['embeddings'][idx1]},{total_df['embeddings'][idx2]},{cosine_sim_val}")


In [ ]:
def make_graph(data, cos_sim=0.9, plot_graph=False):
    edges_subset=list(combinations(data.index, 2))
    edge_sim_labelled=[]

    for idx1, idx2 in tqdm(edges_subset):

        cosine_sim_val=cosine_similarity( [ data['embeddings'][idx1] ], [ data['embeddings'][idx2] ])

        if cosine_sim_val>cos_sim:
            edge_sim_labelled.append([idx1, idx2])
    
    G = nx.Graph()
    G.add_nodes_from(data.index)
    G.add_edges_from(edge_sim_labelled)

    if plot_graph:
        plt.figure(figsize=(20,20))
        nx.draw(G)
    return G

In [ ]:
def make_N2V_embeddings(G, cos_sim=0.9, embedding_size=512, walk_length=10, num_walks=80, window=10, connect_type='sim_country_year'):
    filename=f"emb{embedding_size}_wklen{walk_length}_nwalk{num_walks}_win{window}_{connect_type}"
    if connect_type.find('sim'):
        filename+=f"_csim{cos_sim}"
    filename+=f'_GLen{len(G.nodes)}'
    node2vec = Node2Vec(G, dimensions=embedding_size, walk_length=walk_length,num_walks=num_walks, workers=4)
    node2vec_model = node2vec.fit(window=window, min_count=1,batch_words=4)

    embeddings_map = node2vec_model.wv
    embeddings = embeddings_map[[i for i in range(len(G.nodes))]]

    node2vec_model.save(filename+'.model')

    return embeddings

In [ ]:
def load_N2V_embeddings(G_len, embedding_path='/content/drive/MyDrive/Dataset/UNGDC/emb512_wklen30_nwalk160_win20_cossim0.9_sim.model'):
    node2vec_model=Word2Vec.load(embedding_path)
    embeddings = node2vec_model.wv[[i for i in range(G_len)]]
    return embeddings

In [ ]:
embedding_size=512
walk_length=30
num_walks=160
window=20
connect_type='sim'
# G=make_graph(labelled_cleaned, cos_sim=0.9, plot_graph=False)
# embeddings=make_N2V_embeddings(embedding_size=embedding_size, walk_length=walk_length, num_walks=num_walks, window=window, connect_type=connect_type)

In [ ]:
# print(len(G.nodes))

In [ ]:
# embeddings=load_N2V_embeddings(G)
embeddings=load_N2V_embeddings(2649, '/content/drive/MyDrive/Dataset/UNGDC/emb512_wklen30_nwalk160_win20_sim.model')

In [ ]:
labels={'security': 0, 
        'greeting': 1, 
        'development': 2, 
        'democracy': 3, 
        'human': 4, 
        'un': 5}

train_labels_one_hot=tf.one_hot(
    [ labels[i] for i in labelled_cleaned['coding'].values ],
    len(labels)).numpy()

In [ ]:
X_train, X_eval, y_train, y_eval = train_test_split(embeddings, train_labels_one_hot)

In [ ]:
class Classifier(tf.keras.Model):
    def __init__(self, hidden_dims, output_size, dropout_rate):
        super(Classifier, self).__init__()
        self.dense1 = tf.keras.layers.Dense(hidden_dims[0], activation='relu')
        self.dense2 = tf.keras.layers.Dense(hidden_dims[1], activation='relu')
        self.dense3 = tf.keras.layers.Dense(hidden_dims[2], activation='relu')
        self.dense4 = tf.keras.layers.Dense(output_size, activation='softmax')
        self.bnorm1 = tf.keras.layers.BatchNormalization()
        self.bnorm2 = tf.keras.layers.BatchNormalization()
        self.bnorm3 = tf.keras.layers.BatchNormalization()
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x):
        # x = self.dense1(x)
        # x = self.dropout(x)
        # x = self.bnorm1(x)

        x = self.dense2(x)
        x = self.dropout(x)
        # x = self.bnorm2(x)
        
        x = self.dense3(x)
        x = self.dropout(x)
        # x = self.bnorm3(x)
        
        output = self.dense4(x)
        return output

In [ ]:
tf_model = Classifier((512, 256, 128, 64), len(labels), 0.3)
tf_model.build(input_shape=(None, 512))
tf_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tfa.metrics.F1Score(num_classes=len(labels))])
tf_model.summary()
his = tf_model.fit(X_train, y_train, validation_data=(X_eval, y_eval), epochs=400, verbose=1, 
                           callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10))

Model: "classifier_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             multiple                  0 (unused)
                                                                 
 dense_9 (Dense)             multiple                  131328    
                                                                 
 dense_10 (Dense)            multiple                  32896     
                                                                 
 dense_11 (Dense)            multiple                  774       
                                                                 
 batch_normalization_6 (Batc  multiple                 0 (unused)
 hNormalization)                                                 
                                                                 
 batch_normalization_7 (Batc  multiple                 0 (unused)
 hNormalization)                                      

In [ ]:
tf_model.evaluate(x=X_train, y=y_train)

63/63 [==============================] - 0s 4ms/step - loss: 1.4571 - f1_score: 0.1256


[1.4571446180343628,
 array([0.58327276, 0.        , 0.        , 0.        , 0.        ,
        0.17054263], dtype=float32)]

In [ ]:
tf_model.evaluate(x=X_eval, y=y_eval)

21/21 [==============================] - 0s 4ms/step - loss: 1.4564 - f1_score: 0.1154


[1.4563833475112915,
 array([0.57579064, 0.        , 0.        , 0.        , 0.        ,
        0.11678832], dtype=float32)]

In [ ]:
dims= [(512, 256, 128), (256, 128, 64), (128, 64, 32), (512, 128, 64)]
lrs= [0.5, 0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001]
result={}

In [ ]:
for lr in lrs:
  tf_model = Classifier((512, 256, 128, 64), len(labels), 0.3)
  tf_model.build(input_shape=(None, 512))
  tf_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=lr),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tfa.metrics.F1Score(num_classes=len(labels))])
  # tf_model.summary()
  his = tf_model.fit(X_train, y_train, validation_data=(X_eval, y_eval), epochs=400, verbose=0, 
                            callbacks = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10))
  result[lr]={'loss': his.history['loss'][-1], 'f1': his.history['f1_score'][-1], 'val_loss': his.history['val_loss'][-1],'val_f1':his.history['val_f1_score'][-1]}

In [ ]:
list(result.items())[0][1]['f1']

array([0.58483756, 0.        , 0.        , 0.        , 0.        ,
       0.13623978], dtype=float32)

In [ ]:
print(sorted(list(result.items()), key=lambda k: k[1]['loss'])[0][0])
print(sorted(list(result.items()), key=lambda k: sum(k[1]['f1']), reverse=True)[0][0]) 
print(sorted(list(result.items()), key=lambda k: k[1]['val_loss'])[0][0])
print(sorted(list(result.items()), key=lambda k: sum(k[1]['val_f1']), reverse=True)[0][0])

0.1
0.005
0.005
0.05


In [ ]:
result

{0.5: {'loss': 1.4516513347625732,
  'f1': array([0.58483756, 0.        , 0.        , 0.        , 0.        ,
         0.13623978], dtype=float32),
  'val_loss': 1.4663450717926025,
  'val_f1': array([0.5772532, 0.       , 0.       , 0.       , 0.       , 0.       ],
        dtype=float32)},
 0.1: {'loss': 1.4480030536651611,
  'f1': array([0.58393633, 0.        , 0.00380952, 0.        , 0.        ,
         0.13513513], dtype=float32),
  'val_loss': 1.4495762586593628,
  'val_f1': array([0.5791757 , 0.        , 0.        , 0.        , 0.        ,
         0.10606061], dtype=float32)},
 0.05: {'loss': 1.449190616607666,
  'f1': array([0.58631927, 0.        , 0.        , 0.        , 0.        ,
         0.15508023], dtype=float32),
  'val_loss': 1.4518345594406128,
  'val_f1': array([0.5782609 , 0.        , 0.        , 0.        , 0.        ,
         0.11940298], dtype=float32)},
 0.01: {'loss': 1.4548940658569336,
  'f1': array([0.5843921 , 0.        , 0.        , 0.        , 0.      

In [ ]:
tf_model = Classifier((512, 256, 128, 64), len(labels), 0.5)
tf_model.build(input_shape=(None, 512))
tf_model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tfa.metrics.F1Score(num_classes=len(labels))])
tf_model.summary()
his = tf_model.fit(X_train, y_train, validation_data=(X_eval, y_eval), epochs=400, verbose=1)

In [ ]:
tf_model.evaluate(x=X_train, y=y_train)

63/63 [==============================] - 0s 3ms/step - loss: 1.4520 - f1_score: 0.1256


[1.4519602060317993,
 array([0.58327276, 0.        , 0.        , 0.        , 0.        ,
        0.17054263], dtype=float32)]

In [ ]:
tf_model.evaluate(x=X_eval, y=y_eval)

21/21 [==============================] - 0s 3ms/step - loss: 1.4521 - f1_score: 0.1154


[1.4521138668060303,
 array([0.57579064, 0.        , 0.        , 0.        , 0.        ,
        0.11678832], dtype=float32)]

In [ ]:
max_iter=400
lr=0.1

clf = MLPClassifier(random_state=42, max_iter=max_iter, hidden_layer_sizes=(128, 64, 128), verbose=True, learning_rate_init=lr)
clf.fit(X_train, y_train)

Iteration 1, loss = 3.54012022
Iteration 2, loss = 2.36101974
Iteration 3, loss = 2.34510596
Iteration 4, loss = 2.33624125
Iteration 5, loss = 2.33192501
Iteration 6, loss = 2.33299826
Iteration 7, loss = 2.34266374
Iteration 8, loss = 2.33640629
Iteration 9, loss = 2.33380964
Iteration 10, loss = 2.33413329
Iteration 11, loss = 2.32968202
Iteration 12, loss = 2.32804512
Iteration 13, loss = 2.33023601
Iteration 14, loss = 2.32652949
Iteration 15, loss = 2.32823680
Iteration 16, loss = 2.32580251
Iteration 17, loss = 2.32814320
Iteration 18, loss = 2.32933218
Iteration 19, loss = 2.32613628
Iteration 20, loss = 2.32529994
Iteration 21, loss = 2.32724155
Iteration 22, loss = 2.32729323
Iteration 23, loss = 2.33238476
Iteration 24, loss = 2.33504465
Iteration 25, loss = 2.33223044
Iteration 26, loss = 2.32819780
Iteration 27, loss = 2.32877927
Iteration 28, loss = 2.32474963
Iteration 29, loss = 2.32282855
Iteration 30, loss = 2.32509236
Iteration 31, loss = 2.32733379
Iteration 32, los

MLPClassifier(hidden_layer_sizes=(128, 64, 128), learning_rate_init=0.1,
              max_iter=400, random_state=42, verbose=True)

In [ ]:
clf.score(embeddings, train_labels_one_hot)

In [ ]:
clf.score(X_eval, y_eval)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
preds = clf.predict(embeddings)

In [ ]:
filename = "/content/drive/MyDrive/Dataset/UNGDC/MLP_iter300_layer1024_512_128_sim_country_year.pickle"

# save model
pickle.dump(clf, open(filename, "wb"))

# load model
loaded_model = pickle.load(open(filename, "rb"))

In [ ]:
preds = loaded_model.predict(embeddings)
loaded_model.score(embeddings, train_labels_one_hot)